# CS461 Homework Assignment 3

# Part 1: Solving Cliff Walking with Value Iteration and Policy Iteration
In this part, you will implement **Value Iteration** and **Policy Iteration** methods to solve the `CliffWalking` environment from the Gymnasium library. The goal is to find an optimal policy that minimizes the total cost while avoiding the cliff.

### Learning Objectives
- Understand and implement core MDP methods (Value Iteration and Policy Iteration).
- Visualize and analyze the convergence of state values and policies.
- Compare the performance of both methods in terms of iterations and computation time.

## Setup
### Install Required Libraries
To begin, ensure you have the required packages installed. Run the following cells to install dependencies.

In [ ]:
!pip install swig --quiet

In [ ]:
!pip install gymnasium[all] matplotlib --quiet

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from gymnasium.envs.toy_text import CliffWalkingEnv
import gymnasium as gym
from tqdm import tqdm
import seaborn as sns
from collections import defaultdict
from matplotlib.patches import Patch
from mpl_toolkits.mplot3d import Axes3D


%matplotlib inline

## Environment
The `CliffWalkingEnv` is a 4x12 grid-world environment. The agent starts at the top-left corner and aims to reach the top-right corner. Falling into the cliff results in a significant penalty. The environment has four possible actions:
- **0**: Move up
- **1**: Move right
- **2**: Move down
- **3**: Move left

### Example Render:
S = Start, G = Goal, C = Cliff

In [ ]:
import numpy as np
from gymnasium import Wrapper

class CliffEnvWrapper(Wrapper):
    def __init__(self, env):
        super().__init__(env)
        self.start_state = (3, 0)  # Assumes a fixed start state
        self.terminal_state = (self.env.shape[0] - 1, self.env.shape[1] - 1)  # Bottom-right corner
        self.start_state_index = np.ravel_multi_index(self.start_state, self.env.shape)
        self.terminal_state_index = np.ravel_multi_index(self.terminal_state, self.env.shape)

        # Expose `P` as a direct attribute
        self.P = env.P

        # Correct the transition probabilities
        self.correct_transitions()

    def correct_transitions(self):
        """Corrects the environment's transition probabilities for cliffs and terminal states."""
        for state in range(self.env.nS):
            position = np.unravel_index(state, self.env.shape)

            for action in range(self.env.nA):
                transitions = self.env.P[state][action]

                # If the state is a cliff, reset to the start state with penalty
                if self.env._cliff[position]:
                    self.env.P[state][action] = [(1.0, self.start_state_index, -100, False)]
                    continue

                # If the state is terminal, transitions should only lead to itself
                if position == self.terminal_state:
                    self.env.P[state][action] = [(1.0, state, 0, True)]
                    continue

                # Otherwise, keep the original transition logic
                new_transitions = []
                for prob, next_state, reward, done in transitions:
                    next_position = np.unravel_index(next_state, self.env.shape)
                    # Ensure terminal state is correctly handled
                    if next_position == self.terminal_state:
                        new_transitions.append((prob, next_state, reward, True))
                    else:
                        new_transitions.append((prob, next_state, reward, False))
                self.env.P[state][action] = new_transitions

    def step(self, action):
        """Override step to ensure consistency with wrapped transitions."""
        state, reward, done, truncated, info = super().step(action)

        # If the agent steps into a cliff, reset it to the start state
        position = np.unravel_index(self.env.s, self.env.shape)
        if self.env._cliff[position]:
            self.env.s = self.start_state_index  # Reset to start state
            return self.env.s, reward, done, truncated, info

        return state, reward, done, truncated, info


In [ ]:
env = CliffWalkingEnv(render_mode="rgb_array")
wrapped_env = CliffEnvWrapper(env)

# Use the wrapped environment
observation, info = wrapped_env.reset()
wrapped_env.render()

## Task 1: Implement Value Iteration
Complete the function `value_iteration` below. This method computes the optimal state-value function and policy by iteratively updating state values using the Bellman optimality equation.

In [ ]:
# Value Iteration function skeleton
def value_iteration(env, gamma=0.99, theta=1e-6):
    """
    Performs Value Iteration to compute the optimal value function and policy.
    Args:
        env: The CliffWalking environment.
        gamma: Discount factor.
        theta: Threshold for convergence.
    Returns:
        V: Optimal state-value function.
        policy: Optimal policy.
    """
    V = np.zeros(env.observation_space.n)
    while True:
        delta = 0
        # TODO: Update state values using Bellman optimality equation

        if delta < theta:
            break

    policy = np.zeros(env.observation_space.n, dtype=int)
    # TODO: Derive optimal policy from the value function

    return V, policy

## Task 2: Implement Policy Iteration
Complete the function `policy_iteration` below. This method alternates between policy evaluation and policy improvement until the policy converges to the optimal policy.

In [ ]:
# Policy Iteration function skeleton
def policy_iteration(env, gamma=0.99):
    """
    Performs Policy Iteration to compute the optimal value function and policy.
    Args:
        env: The CliffWalking environment.
        gamma: Discount factor.
    Returns:
        V: Optimal state-value function.
        policy: Optimal policy.
    """
    policy = np.zeros(env.observation_space.n, dtype=int)
    V = np.zeros(env.observation_space.n)

    while True:
        # TODO: Perform policy evaluation and policy improvement
        break # replace with your actual algorithm
        # Check for policy convergence

    return V, policy

## Visualization
To help you debug and understand the algorithms, use the following function to visualize the state values.

In [ ]:
def visualize_values(values, grid_size=(4, 12)):
    plt.figure(figsize=(12, 4))
    grid_values = np.array(values).reshape(grid_size)
    plt.imshow(grid_values, cmap='coolwarm', interpolation='nearest')
    for (i, j), val in np.ndenumerate(grid_values):
        plt.text(j, i, f"{val:.2f}", ha='center', va='center', color='black')
    plt.title("State Values")
    plt.colorbar(label="Value")
    plt.show()

In [ ]:
# Test the solutions and visualize results
# Value Iteration

wrapped_env.reset()
V_vi, policy_vi = value_iteration(wrapped_env)
visualize_values(V_vi)


# Policy Iteration
V_pi, policy_pi = policy_iteration(env)
visualize_values(V_pi)


## Task 3: Analysis

Briefly answer the questions below. You may write your answers after "Answer:".

1. Compare the number of iterations and computation time for Value Iteration and Policy Iteration.

Answer:

2. Visualize the state values and return the optimal policy for both methods and discuss any differences.

Answer:

3. Experiment with different values of `gamma` and observe how it affects convergence and the optimal policy.

Answer:

4. If the grids become "slippery," where actions might lead to unintended moves, how would the optimal policy change?

Answer:

5. If falling into the cliff results in a reward of -100, why is the state value not exactly -100?

Answer:

# Part 2: Reinforcement Learning

In this part, you will implement two reinforcement learning algorithms, Q-learning and SARSA, for Blackjack.

Blackjack, or 21, is a casino card game where players aim to beat the dealer by having a hand value closer to 21 without exceeding it. Cards 2-10 are worth their face value, face cards are 10, and Aces are 1 or 11 (whichever is more beneficial). Players can "hit" for more cards or "stand" to keep their total, while the dealer follows fixed rules.

# Setup

Let's create the game environment first. In Gymnasium's Blackjack environment, the cards are drawn from an infinite deck (i.e. drawn with replacement). This means that exactly which cards are drawn is not important except an ace (since it can have two values); only the total values are enough.

Below, you can see that the observation space size is 32 \* 11 \* 2, where 32 is the number of total values the player can have over the course of the game, 11 is the numbers dealer's value can take, and 2 is whether the player has an usable ace or not. The player has only 2 actions, hit (1) or stand (0). A reward of 1 or -1 are given when the player wins or loses, respectively.

In [ ]:
env = gym.make('Blackjack-v1', natural=False, sab=False, render_mode="rgb_array")

obs, info = env.reset()

env.render()
print("Observation space:", env.observation_space)
print("Action space:", env.action_space)

## Task 4: Q-learning

In class, you have learned Q-learning, which uses the following updates to learn Q-values:

$Q(s_t, a_t) \gets Q(s_t, a_t) + \alpha (r_t + \gamma \max_{a'} Q(s_{t+1}, a') - Q(s_t, a_t))$

In the function below, implement the Q-learning algorithm with $\epsilon$-greedy exploration with exponential decay.

In [ ]:
def q_learning(env, num_episodes, gamma=0.9, alpha=0.01, epsilon_start=1.0, epsilon_min=0.1, epsilon_decay=0.995):
    # Initialize Q-table
    q_table = np.zeros((32, 11, 2, env.action_space.n))
    policy_table = np.zeros((32, 11, 2))

    epsilon = epsilon_start  # Initialize epsilon value

    """
    Learn Q-values and the policy using Q-learning update rule.

    Inputs:
        - num_episodes: Number of episodes to run the algorithm for.
        - gamma: Discount factor for future rewards.
        - alpha: Learning rate for updating Q-values.
        - epsilon_start: Initial value for epsilon in the epsilon-greedy policy.
        - epsilon_min: Minimum value of epsilon.
        - epsilon_decay: Decay factor for epsilon.

    Returns:
        - q_table: Q-values for each state-action (or observation-action) pair.
        - policy_table: Optimal policy with respect to the learned Q-values.
    """

    # START CODE HERE

    # END CODE HERE

    return q_table, policy_table


# Visualizing the results

Now we can visualize what the algorithm has learned.

In [ ]:
def evaluate_policy(q_table, policy_table):
    """
    Evaluate the value of each state under a given policy.

    Parameters:
    - q_table: The Q-table learned from Q-learning (shape: [player_sum, dealer_card, usable_ace, action]).
    - policy_table: The policy table derived from the Q-table (shape: [player_sum, dealer_card, usable_ace]).

    Returns:
    - value_table: The value of each state under the given policy (shape: [player_sum, dealer_card, usable_ace]).
    """
    # Initialize the value table
    value_table = np.zeros(policy_table.shape)

    # Iterate over all possible states
    for player_sum in range(32):
        for dealer_card in range(11):
            for usable_ace in range(2):
                # Get the action chosen by the policy for this state
                action = policy_table[player_sum, dealer_card, usable_ace]

                # Get the Q-value for the chosen action from q_table
                value_table[player_sum, dealer_card, usable_ace] = q_table[player_sum, dealer_card, usable_ace, action]

    return value_table


In [ ]:
def plot_value_surface(value_table, usable_ace):
    """
    Plot the 3D surface for the value table under a given condition (usable ace or not).
    Only considers player sums >= 12.

    Parameters:
    - value_table: The value table computed from the policy.
    - usable_ace: Boolean, whether to visualize values for states with a usable ace (True) or without (False).
    """
    ace_index = 1 if usable_ace else 0
    # Focus on valid player sums (12-21) and dealer card values (1-10)
    player_sums = np.arange(12, 22)  # Player sum values: 12 to 21
    dealer_cards = np.arange(1, 11)  # Dealer card values: 1 to 10

    # Prepare the meshgrid for plotting
    player_sum_grid, dealer_card_grid = np.meshgrid(player_sums, dealer_cards)

    # Extract the value table slice for usable ace or not (only for player sums >= 12)
    z_values = value_table[12:22, 1:11, ace_index]  # (10, 10) shape

    # Plotting
    fig = plt.figure(figsize=(12, 8))
    ax = fig.add_subplot(111, projection='3d')

    # Transpose z_values to match the meshgrid dimensions
    ax.plot_surface(dealer_card_grid, player_sum_grid, z_values.T, cmap='viridis')

    # Labels and title
    title = "Value Surface (Usable Ace)" if usable_ace else "Value Surface (No Usable Ace)"
    ax.set_title(title)
    ax.set_ylabel('Player Sum')
    ax.set_xlabel('Dealer Card')
    ax.set_zlabel('State Value')

    plt.show()


def plot_policy_heatmap(policy_table, usable_ace):
    """
    Plot the policy heatmap for usable ace and non-usable ace conditions.
    Only considers player sums >= 12.

    Parameters:
    - policy_table: The policy table computed from Q-learning.
    - usable_ace: Boolean, whether to visualize for states with usable ace (True) or not (False).
    """
    ace_index = 1 if usable_ace else 0
    # Focus on valid player sums (12-21) and dealer card values (1-10)
    player_sums = np.arange(12, 22)  # Player sum values: 12 to 21
    dealer_cards = np.arange(1, 11)  # Dealer card values: 1 to 10

    # Extract the policy table slice for usable ace or not (only for player sums >= 12)
    policy_values = policy_table[12:22, 1:11, ace_index]  # (10, 10) shape

    # Plotting the heatmap
    plt.figure(figsize=(12, 8))
    sns.heatmap(policy_values, annot=True, fmt="d", cmap="coolwarm", xticklabels=dealer_cards, yticklabels=player_sums, cbar=False)

    # Labels and title
    title = "Policy Heatmap (Usable Ace)" if usable_ace else "Policy Heatmap (No Usable Ace)"
    plt.title(title)
    plt.xlabel('Dealer Card')
    plt.ylabel('Player Sum')

    plt.show()


In [ ]:
q_table, policy_table = q_learning(env, num_episodes=100000)

In [ ]:
value_table = evaluate_policy(q_table, policy_table)

plot_value_surface(value_table, usable_ace=True)
plot_value_surface(value_table, usable_ace=False)

In [ ]:
plot_policy_heatmap(policy_table, usable_ace=True)
plot_policy_heatmap(policy_table, usable_ace=False)

# Task 5: Analysis of the results

Briefly answer the questions below. You may write your answers in this cell right after "Answer:".

1. Is Q-learning on-policy or off-policy? Why?

Answer:

2. What can you infer from the visualization of the value function (using the default hyperparameters for training)? Does it match your intuition?

Answer:

3. Intuitively, does the policy seem to be optimal (using the default hyperparameters for training)? Why or why not?

Answer:

4. For both value and policy visualizations, what difference does having or not having an usable ace make?

Answer:

5. Experiment with different values of $\gamma, \epsilon_{start}, \epsilon_{min}$, $\epsilon$ decay factor (you should also try a decay of 1, which corresponds to vanilla $\epsilon$-greedy with $\epsilon=\epsilon_{start}$). How do these hyperparameters affect your results?

Answer:

# Task 6: SARSA
SARSA is another popular reinforcement learning algorithm. It uses the following update rule:

$Q(s_t, a_t) \gets Q(s_t, a_t) + \alpha (r_t + \gamma Q(s_{t+1}, a_{t+1}) - Q(s_t, a_t))$

where $a_{t+1}$ is also selected using the $\epsilon$-greedy policy that is used to select $a_t$.

Now, implement SARSA with $\epsilon$-greedy exploration with decay in the function body below, as before.

In [ ]:
def sarsa(env, num_episodes, gamma=0.9, alpha=0.01, epsilon_start=1.0, epsilon_min=0.1, epsilon_decay=0.995):
    # Initialize Q-table
    q_table = np.zeros((32, 11, 2, env.action_space.n))
    policy_table = np.zeros((32, 11, 2))

    epsilon = epsilon_start

    """
    Learn Q-values and the policy using SARSA update rule.

    Inputs:
        - num_episodes: Number of episodes to run the algorithm for.
        - gamma: Discount factor for future rewards.
        - alpha: Learning rate for updating Q-values.
        - epsilon_start: Initial value for epsilon in the epsilon-greedy policy.
        - epsilon_min: Minimum value of epsilon.
        - epsilon_decay: Decay factor for epsilon.

    Returns:
        - q_table: Q-values for each state-action (or observation-action) pair.
        - policy_table: Optimal policy with respect to the learned Q-values.
    """
    # START CODE HERE

    # END CODE HERE

    return q_table, policy_table


In [ ]:
q_table, policy_table = sarsa(env, num_episodes=100000)

# Visualizing the results

In [ ]:
value_table = evaluate_policy(q_table, policy_table)

plot_value_surface(value_table, usable_ace=True)
plot_value_surface(value_table, usable_ace=False)

In [ ]:
plot_policy_heatmap(policy_table, usable_ace=True)
plot_policy_heatmap(policy_table, usable_ace=False)

# Task 7: Analysis of results

Briefly answer the questions below.

1. Is SARSA on-policy or off-policy? Why?

Answer:

2. Experiment with different values of $\gamma, \epsilon_{start}, \epsilon_{min}$, $\epsilon$ decay factor (you should also try a decay of 1, which corresponds to vanilla $\epsilon$-greedy with $\epsilon=\epsilon_{start}$). How do these hyperparameters affect your results?

Answer: